## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key

from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chuy,UY,-33.6971,-53.4616,59.58,67,98,11.88,overcast clouds
1,1,Paita,PE,-5.0892,-81.1144,81.16,70,13,16.24,few clouds
2,2,Richards Bay,ZA,-28.7830,32.0377,60.35,73,0,5.21,clear sky
3,3,Oussouye,SN,12.4850,-16.5469,87.98,82,100,6.76,overcast clouds
4,4,Mataura,NZ,-46.1927,168.8643,37.47,91,6,3.02,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
best_temp_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & (city_data_df["Max Temp"]>=min_temp)]
best_temp_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Paita,PE,-5.0892,-81.1144,81.16,70,13,16.24,few clouds
3,3,Oussouye,SN,12.4850,-16.5469,87.98,82,100,6.76,overcast clouds
8,8,Bethel,US,41.3712,-73.4140,80.02,76,40,1.01,scattered clouds
9,9,Albany,US,42.6001,-73.9662,80.85,72,72,10.31,broken clouds
12,12,Tabas,IR,33.5959,56.9244,84.65,12,1,4.65,clear sky
14,14,Mitsamiouli,KM,-11.3847,43.2844,77.29,78,20,2.30,few clouds
20,20,Hithadhoo,MV,-0.6000,73.0833,82.11,74,100,17.00,light rain
22,22,Bontang,ID,0.1333,117.5000,75.85,87,100,1.77,overcast clouds
28,28,Naze,JP,28.3667,129.4833,75.90,77,3,6.60,clear sky
35,35,Sao Filipe,CV,14.8961,-24.4956,79.21,80,36,7.07,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
best_temp_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
best_temp_cities_df.count()

City_ID                213
City                   213
Country                211
Lat                    213
Lng                    213
Max Temp               213
Humidity               213
Cloudiness             213
Wind Speed             213
Current Description    213
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
cleaned_best_temp_cities_df = best_temp_cities_df.dropna()

In [8]:
cleaned_best_temp_cities_df.count()

City_ID                211
City                   211
Country                211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = cleaned_best_temp_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"

hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Paita,PE,81.16,few clouds,-5.0892,-81.1144,NaN
3,Oussouye,SN,87.98,overcast clouds,12.4850,-16.5469,NaN
8,Bethel,US,80.02,scattered clouds,41.3712,-73.4140,NaN
9,Albany,US,80.85,broken clouds,42.6001,-73.9662,NaN
12,Tabas,IR,84.65,clear sky,33.5959,56.9244,NaN
14,Mitsamiouli,KM,77.29,few clouds,-11.3847,43.2844,NaN
20,Hithadhoo,MV,82.11,light rain,-0.6000,73.0833,NaN
22,Bontang,ID,75.85,overcast clouds,0.1333,117.5000,NaN
28,Naze,JP,75.90,clear sky,28.3667,129.4833,NaN
35,Sao Filipe,CV,79.21,scattered clouds,14.8961,-24.4956,NaN


In [10]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found...skipping")
        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [11]:
hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Paita,PE,81.16,few clouds,-5.0892,-81.1144,Hotel Náutico de Paita
3,Oussouye,SN,87.98,overcast clouds,12.4850,-16.5469,Auberge du Routard
8,Bethel,US,80.02,scattered clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
9,Albany,US,80.85,broken clouds,42.6001,-73.9662,NaN
12,Tabas,IR,84.65,clear sky,33.5959,56.9244,شرکت معماری مهراز شایگان آراد


In [12]:
hotel_df.isnull().sum()


City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             10
dtype: int64

In [13]:
# 7. Drop the rows where there is no Hotel Name.
cleaned_hotel_df = hotel_df.dropna()


In [14]:
cleaned_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Paita,PE,81.16,few clouds,-5.0892,-81.1144,Hotel Náutico de Paita
3,Oussouye,SN,87.98,overcast clouds,12.4850,-16.5469,Auberge du Routard
8,Bethel,US,80.02,scattered clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
12,Tabas,IR,84.65,clear sky,33.5959,56.9244,شرکت معماری مهراز شایگان آراد
14,Mitsamiouli,KM,77.29,few clouds,-11.3847,43.2844,Foyer ADM
20,Hithadhoo,MV,82.11,light rain,-0.6000,73.0833,Scoop Guest House
22,Bontang,ID,75.85,overcast clouds,0.1333,117.5000,KOPKAR Hotel Bintang Sintuk
28,Naze,JP,75.90,clear sky,28.3667,129.4833,オーベルジュ・アガルイ
35,Sao Filipe,CV,79.21,scattered clouds,14.8961,-24.4956,Hotel Xaguate
36,San Patricio,US,86.09,scattered clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
cleaned_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in cleaned_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = cleaned_hotel_df[["Lat", "Lng"]]


In [17]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
max_temp = cleaned_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))